In [3]:
import requests
import pandas as pd
import numpy as np
import matplotlib as plt
import plotly.express as px
# For plotly to work you also need to install the statsmodels package as well

# Get live data from FPL site. Not necessary to do every time. 
# TODO: Write to a file and pull from that instead
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

# Uncomment to see all columns
# elements_df.columns

# Adding .copy() operation to avoid hidden chaining and ensuring data integrity
slim_elements_df = elements_df[['first_name','second_name', 'team', 'element_type','selected_by_percent','now_cost','minutes','total_points', 'value_season']].copy()

# Change from positional values to their actual names
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

# Replace team integer IDs with actual team names
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

# Add value column as a float type, so we can then use it fo
slim_elements_df['value'] = slim_elements_df.value_season.astype(float)

slim_elements_df = slim_elements_df.loc[slim_elements_df.value > 0]
slim_elements_df = slim_elements_df.loc[slim_elements_df.minutes > 500]

# TODO: Divide points by minutes * 90 to get points per 90, that over cost is better than value. minutes can be size
# TODO: Drop Stuart Dallas and anyone injured

# Combinging first and last name into a single column
slim_elements_df['name'] = slim_elements_df['first_name'] + ' ' + slim_elements_df['second_name']

# Convert now_cost into cost, make it a decimal
slim_elements_df['cost'] = (slim_elements_df.now_cost.astype(float) / 10)

# Create a points per 90 minutes column, rounding to 2 decimal places
slim_elements_df['points_per_90'] = (slim_elements_df['total_points'] / slim_elements_df['minutes'] * 90).round(2)

# Creating a final dataframe with only the elements we want again

fpl_players = slim_elements_df[['name', 'position', 'minutes', 'cost', 'total_points', 'value', 'points_per_90']]
#fpl_players.head(10)

# Generating position specific tables if you want
# mid_df = fpl_players.loc[slim_elements_df.position == 'Midfielder']
# Output a table of the top 35 midfielders by points per 90
#mid_df.sort_values(by=['points_per_90'],ascending=False).head(35)



# Make a scatter graph of points per 90 by cost

value_plot = px.scatter(fpl_players,
                           x='cost',
                           y='points_per_90',
                           size='total_points',
                           trendline="ols",
                           color='position',
                           hover_data=['name'])
value_plot.write_html("/home/adam/jupyter/fpl_value_plot.html")
value_plot.show()
